 EPIC, the Extended PID Configurator, Ihr vertrauenswürdiger Konfigurator :)
V0.1 := Initial release.

#Writed according PEP8 style guide
#https://realpython.com/python-pep8/

In [97]:
#Libraries import
import os
import time
import datetime
import pandas as pd
import numpy as np
from numpy import random
import scipy.sparse as sp

##Version information
print(f"Pandas version: {pd.__version__}") #Pandas version number
print(f"Numpy version: {np.__version__}") #numpy version number



Pandas version: 1.4.2
Numpy version: 1.21.5


In [98]:
#Global variables
import_mode = True # import mode (True/False) - Imports an csv from PID
verbose_mode = False # verbose mode (True/False)
debug_mode = True # debug mode (True/False)
random_mode = False # random data generation  (True/False)
simulation_mode = False # simulation (True/False)

In [99]:
#Generate standard templated
generate_tag_number_definitions = False #Every element has a description and a unique number for software modules
#generate_element_list_alarm_matrix = False #Every element has a unique number of alarms
#generate_element_list_hardware_matrix = False #All possible combinations of elements have a unique IO

In [100]:
#Data generation
generate_element_list = False
generate_functional_matrix = False
generate_alarm_matrix = False
generate_hardware_config = False
generate_symbolic   = False
generate_s_plant    = False
generate_s_line = False
generate_flow_chart = False


In [101]:
if import_mode: 
    ##Change working directory to temp/epic
    os.chdir('C://temp/epic/')
    cwd = os.getcwd()
    print('Current Working Directory is: ', cwd)
    


Current Working Directory is:  C:\temp\epic


In [102]:
if import_mode:
    os.listdir(cwd) #List directory

#Random_mode:
Generate aleatories PID numbers and symbols.



In [103]:
#Generate list of X random numbers from 1.100 - 9.999:
if random_mode:
    random_PID_numbers=list(random.randint(1100,10000, size=(1000)))

    if verbose_mode:    print(random_PID_numbers)

In [104]:
#Generate PID symbol
if random_mode:
    random_PID_symbols = list(random.choice(['B', 'C', 'D', 'EC', 'F', 'FA-', 'FIA+', 'FICA+', 'FSA-', 'GIA', 'GSA+', 'GSA-', 'GZA-', 'H', 'K', 'L', 'LSA-', 'M',
    'MIA±', 'NC', 'P', 'PDI', 'PDICA±', 'PDIS', 'PI', 'PSA-', 'QIA+', 'QIA-', 'QSA+', 'S', 'SC', 'SIA±', 'T', 'TI', 'TIC', 'TICA±', 'TSA+',
    'V', 'W', 'WICA±', 'XA', 'Y', 'YC'], size=(1000)))

    if verbose_mode:    print(random_PID_symbols)
    if debug_mode:      print(type(random_PID_symbols))

Series and DataFrames

In [105]:
#pid_series = pd.Series(random_PID_symbols, random_PID_numbers)
#pid_series = pid_series.sort_index(axis=0,ascending=True)
#pid_series

#Import Tag number definitions as a pandas series
#tnd = pd.read_csv('TND.csv',names=['PID','Description'])
tnd = pd.read_excel('TND.xlsx',)
tnd

,PID,Description
0,Y.100,Intake air fan
1,Y.101,Filter 1
2,Y.102,Filter 2
3,Y.103,Filter 3
4,Y.104,NaN
...,...,...
895,Y.995,NaN
896,Y.996,NaN
897,Y.997,NaN
898,Y.998,NaN


In [106]:
PID_series = tnd['PID'].squeeze()
pid_exist = PID_series.str.contains('.100')

In [132]:
pid_exist.index
pid_exist.get_loc['True']

AttributeError: 'Series' object has no attribute 'get_loc'

In [107]:
tnd.at[0,'Description']

'Intake air fan'

In [108]:
#return index where PID Y.xxx is located 
tnd.loc[tnd['PID'] == 'Y.101']


,PID,Description
1,Y.101,Filter 1


In [109]:
selected_index = pd.Index(tnd['PID'])
new_output = selected_index.get_loc('Y.101')
print(new_output)

1


In [110]:
tnd.loc[tnd.PID.str.contains('102')]

,PID,Description
2,Y.102,Filter 2


In [111]:
search_index = tnd.loc[tnd.PID.str.contains('102')]
print(search_index)

     PID Description
2  Y.102    Filter 2


Load data from PID (a csv file into pandas)

In [112]:
#df_data_raw = pd.read_csv('PID.csv', encoding = "utf-8", engine='python')
df_data_raw = pd.read_csv('PID.csv', index_col=['Prop_PID_Number'], encoding = "utf-8", engine='python')
df_data_raw.sort_index(axis=0,ascending=True)

,Prop_PID_Element
Prop_PID_Number,
1.100,XA
1.100,SC
1.100,M
1.100,V
1.101,F
...,...
2.304,M
2.308,FSA-
2.350,P


In [113]:
#datatypes of each column
df_data_raw.dtypes

Prop_PID_Element    object
dtype: object

In [114]:
#serie_of_elements = df_data_raw['Prop_PID_Number'].squeeze()
#print(serie_of_elements)
#print (type(serie_of_elements))
#serie_of_elements.unique()

Reading Excel File

We can read an excel file into a data frame:

pd.read_excel('myExcel.xlsx', index_col=['ColumnA'])

We can also load one sheet into a data frame:

pd.read_excel(open('myExcel.xlsx', 'rb'), sheet_name='Sheet1')

In [115]:
df_data_excel = pd.read_excel('TND.xlsx')

In [116]:
df_data_excel.head(1000)

,PID,Description
0,Y.100,Intake air fan
1,Y.101,Filter 1
2,Y.102,Filter 2
3,Y.103,Filter 3
4,Y.104,NaN
...,...,...
895,Y.995,NaN
896,Y.996,NaN
897,Y.997,NaN
898,Y.998,NaN
